In [102]:
import pandas as pd
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import datetime as dt
from textblob import TextBlob

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/chloe_ycl/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [118]:
tweets_list = []
until = dt.datetime.strftime(dt.date.today(), '%Y-%m-%d')
since = dt.datetime.strftime(dt.datetime.strptime(until, '%Y-%m-%d') - dt.timedelta(days=31), '%Y-%m-%d')
# -exclude:retweetedTweet
query = f'$TSLA lang:en until:{until} since:{since} -filter:links -filter:replies -is:retweet -is:nullcast'
n_tweets = 50000

# count -1 mean get all the rows
for i, tweet in tqdm(enumerate(sntwitter.TwitterSearchScraper(query).get_items()), total=n_tweets):
    tweets_list.append([tweet.date, tweet.id, tweet.rawContent, tweet.user, tweet.cashtags])
    if i > n_tweets:
        break
    
#Creating a dataframe from the tweets list above 
df = pd.DataFrame(tweets_list, columns=['date', 'id', 'content', 'user','cashtag'])
df.head(10)

 49%|████▉     | 24594/50000 [14:01<14:28, 29.24it/s]


,date,id,content,user,cashtag
0,2023-03-05 23:47:39+00:00,1632528328765653000,$TSLA loose bolts 🤣🤣come on na,https://twitter.com/freshFranks,[TSLA]
1,2023-03-05 23:47:11+00:00,1632528212881473538,So much negativity on $TSLA these days… probab...,https://twitter.com/WholeMarsBlog,[TSLA]
2,2023-03-05 23:43:54+00:00,1632527385592569856,Hope you all are fucking locked in! \n\n$spy $...,https://twitter.com/New_Highs_,"[spy, qqq, tsla, spx, dxy, vix]"
3,2023-03-05 23:39:18+00:00,1632526227905867777,Updated portfolio 3/5/2023\n\n$NVDA 22%\n$ALGM...,https://twitter.com/meirlandau10,"[NVDA, ALGM, AI, TSLA, DUOL, LNTH, FOUR, MELI,..."
4,2023-03-05 23:33:04+00:00,1632524658829414401,Seeing a lot of $TSLA Model 3 SR+ temp tags. \...,https://twitter.com/GreatPaul_Smith,"[TSLA, TSLA, TSLAQ]"
5,2023-03-05 23:28:17+00:00,1632523457752707080,Stocks I will be watching tomorrow:\n\n$NVDA -...,https://twitter.com/TradingPeacock,"[NVDA, TSLA]"
6,2023-03-05 23:25:24+00:00,1632522731286609922,What is on your watchlist this week?\n\n$NVDA\...,https://twitter.com/ecgreendays,"[NVDA, TSLA, COIN, CVNA, AFRM, VIX]"
7,2023-03-05 23:15:54+00:00,1632520341346983937,"Enjoy the way up bulls… SPX over 5,000 is comi...",https://twitter.com/Mars_Float,"[SPY, TSLA, QQQ, UPRO, BTC.X, AAPL]"
8,2023-03-05 23:13:20+00:00,1632519694422016001,Guess They want Pain back? What a silky road.....,https://twitter.com/calms,"[SPY, TSLA, SPX]"
9,2023-03-05 23:08:52+00:00,1632518568905433088,"From now on, when I hear an analyst refer to $...",https://twitter.com/EBITDAtiger,[TSLA]


In [119]:
df.tail(10)

,date,id,content,user,cashtag
24584,2023-02-03 00:17:18+00:00,1621301766603313152,$TSLA Imagine if Twitter didn't exists and the...,https://twitter.com/Nergock,[TSLA]
24585,2023-02-03 00:14:58+00:00,1621301179228778496,$TSLA One of the satisfaction of owning TSLA i...,https://twitter.com/Nergock,[TSLA]
24586,2023-02-03 00:12:58+00:00,1621300678114131973,$TSLA it's official. Tesla is recession proof....,https://twitter.com/JohnBerHaja,"[TSLA, GOOG, AAPL]"
24587,2023-02-03 00:09:01+00:00,1621299682747256832,Think the fact that $aapl did well for the sal...,https://twitter.com/OptionsScott,"[aapl, Tsla]"
24588,2023-02-03 00:08:43+00:00,1621299608239439873,I have to tell you something…🤓\n\nI rarely do ...,https://twitter.com/bullishmatt,[TSLA]
24589,2023-02-03 00:08:33+00:00,1621299564266536960,$AAPL just ended the rally today $TSLA,https://twitter.com/prayman,"[AAPL, TSLA]"
24590,2023-02-03 00:07:55+00:00,1621299405512118272,MY NEW WEBSITE WILL BE LIVE TOMORROW! 🔥🔥🔥 $TSLA,https://twitter.com/Teslaconomics,[TSLA]
24591,2023-02-03 00:07:16+00:00,1621299241921421312,Bit the bullet and bought a iPhone 14 Pro📱 . D...,https://twitter.com/itechnosmith,"[AAPL, TSLA]"
24592,2023-02-03 00:04:18+00:00,1621298494962106371,⚡️Ticker: $TSLA\n🚀Strike: $195 CALL\n🔮Expiring...,https://twitter.com/OptionsFlowLive,[TSLA]
24593,2023-02-03 00:01:24+00:00,1621297764717170690,I see more bullish news about $tsla lately. Cy...,https://twitter.com/Steven_think,[tsla]


In [120]:
# Create a function to clean the tweets
def clean_text(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('\n', '', text) # Removing \n
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

#applying this function to Text column of our dataframe
df["content"] = df["content"].apply(clean_text)

df.head(5)

,date,id,content,user,cashtag
0,2023-03-05 23:47:39+00:00,1632528328765653000,$TSLA loose bolts 🤣🤣come on na,https://twitter.com/freshFranks,[TSLA]
1,2023-03-05 23:47:11+00:00,1632528212881473538,So much negativity on $TSLA these days… probab...,https://twitter.com/WholeMarsBlog,[TSLA]
2,2023-03-05 23:43:54+00:00,1632527385592569856,Hope you all are fucking locked in! $spy $qqq ...,https://twitter.com/New_Highs_,"[spy, qqq, tsla, spx, dxy, vix]"
3,2023-03-05 23:39:18+00:00,1632526227905867777,Updated portfolio 3/5/2023$NVDA 22%$ALGM 20%$A...,https://twitter.com/meirlandau10,"[NVDA, ALGM, AI, TSLA, DUOL, LNTH, FOUR, MELI,..."
4,2023-03-05 23:33:04+00:00,1632524658829414401,Seeing a lot of $TSLA Model 3 SR+ temp tags. M...,https://twitter.com/GreatPaul_Smith,"[TSLA, TSLA, TSLAQ]"


In [121]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using SentimentIntensityAnalyzer's sentiment method
    '''
    # create SentimentIntensityAnalyzer object of passed tweet text
    # return a dict of results
    SIA_obj = SentimentIntensityAnalyzer()
    analysis = SIA_obj.polarity_scores(tweet)

    # decide sentiment as positive, negative and neutral
    if analysis['neg'] > analysis['pos']:
        return 'negative'
    elif analysis['neg'] < analysis['pos']:
        return 'positive'
    else:
        return 'neutral'

In [122]:
def get_tweet_sentiment_score(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using SentimentIntensityAnalyzer's sentiment method
    '''
    # create SentimentIntensityAnalyzer object of passed tweet text
    # return a dict of results
    SIA_obj = SentimentIntensityAnalyzer()
    analysis = SIA_obj.polarity_scores(tweet)

    # decide sentiment as positive, negative and neutral
    if analysis['neg'] > analysis['pos']:
        return analysis['neg']
    elif analysis['neg'] < analysis['pos']:
        return analysis['pos']
    else:
        return analysis['neu']

In [123]:
df['sentiment'] = df['content'].apply(get_tweet_sentiment)

df['score'] = df['content'].apply(get_tweet_sentiment_score)

df.head(10)

,date,id,content,user,cashtag,sentiment,score
0,2023-03-05 23:47:39+00:00,1632528328765653000,$TSLA loose bolts 🤣🤣come on na,https://twitter.com/freshFranks,[TSLA],negative,0.315
1,2023-03-05 23:47:11+00:00,1632528212881473538,So much negativity on $TSLA these days… probab...,https://twitter.com/WholeMarsBlog,[TSLA],negative,0.245
2,2023-03-05 23:43:54+00:00,1632527385592569856,Hope you all are fucking locked in! $spy $qqq ...,https://twitter.com/New_Highs_,"[spy, qqq, tsla, spx, dxy, vix]",positive,0.210
3,2023-03-05 23:39:18+00:00,1632526227905867777,Updated portfolio 3/5/2023$NVDA 22%$ALGM 20%$A...,https://twitter.com/meirlandau10,"[NVDA, ALGM, AI, TSLA, DUOL, LNTH, FOUR, MELI,...",neutral,1.000
4,2023-03-05 23:33:04+00:00,1632524658829414401,Seeing a lot of $TSLA Model 3 SR+ temp tags. M...,https://twitter.com/GreatPaul_Smith,"[TSLA, TSLA, TSLAQ]",positive,0.097
5,2023-03-05 23:28:17+00:00,1632523457752707080,Stocks I will be watching tomorrow:$NVDA - Fri...,https://twitter.com/TradingPeacock,"[NVDA, TSLA]",positive,0.078
6,2023-03-05 23:25:24+00:00,1632522731286609922,What is on your watchlist this week?$NVDA$TSLA...,https://twitter.com/ecgreendays,"[NVDA, TSLA, COIN, CVNA, AFRM, VIX]",positive,0.062
7,2023-03-05 23:15:54+00:00,1632520341346983937,"Enjoy the way up bulls… SPX over 5,000 is comi...",https://twitter.com/Mars_Float,"[SPY, TSLA, QQQ, UPRO, BTC.X, AAPL]",positive,0.158
8,2023-03-05 23:13:20+00:00,1632519694422016001,Guess They want Pain back? What a silky road.....,https://twitter.com/calms,"[SPY, TSLA, SPX]",negative,0.243
9,2023-03-05 23:08:52+00:00,1632518568905433088,"From now on, when I hear an analyst refer to $...",https://twitter.com/EBITDAtiger,[TSLA],neutral,1.000


In [124]:
df['date']= df['date'].apply(lambda x: x.strftime('%d-%m-%Y'))
df.head(5)

,date,id,content,user,cashtag,sentiment,score
0,05-03-2023,1632528328765653000,$TSLA loose bolts 🤣🤣come on na,https://twitter.com/freshFranks,[TSLA],negative,0.315
1,05-03-2023,1632528212881473538,So much negativity on $TSLA these days… probab...,https://twitter.com/WholeMarsBlog,[TSLA],negative,0.245
2,05-03-2023,1632527385592569856,Hope you all are fucking locked in! $spy $qqq ...,https://twitter.com/New_Highs_,"[spy, qqq, tsla, spx, dxy, vix]",positive,0.210
3,05-03-2023,1632526227905867777,Updated portfolio 3/5/2023$NVDA 22%$ALGM 20%$A...,https://twitter.com/meirlandau10,"[NVDA, ALGM, AI, TSLA, DUOL, LNTH, FOUR, MELI,...",neutral,1.000
4,05-03-2023,1632524658829414401,Seeing a lot of $TSLA Model 3 SR+ temp tags. M...,https://twitter.com/GreatPaul_Smith,"[TSLA, TSLA, TSLAQ]",positive,0.097


In [126]:
df[['day', 'month', 'year']] = df.date.str.split("-", expand=True)
df = df.astype({'day': 'int64', 'month': 'int64', 'year':'int64'})
df.head(5)
df.dtypes

date          object
id             int64
content       object
user          object
cashtag       object
sentiment     object
score        float64
day            int64
month          int64
year           int64
dtype: object

In [ ]:
from google.cloud import bigquery

PROJECT_ID = 'deb-01-372112'
DATASET_ID = 'SIA_Stocks'

# create bigquery client
client = bigquery.Client()

# create table
table_id = "{}.SIA_Stocks.twitter".format(client.project)

schema = [
    bigquery.SchemaField("date", "string", mode="REQUIRED"),
    bigquery.SchemaField("open", "float", mode="REQUIRED"),
    bigquery.SchemaField("high", "float", mode="REQUIRED"),
    bigquery.SchemaField("low", "float", mode="REQUIRED"),
    bigquery.SchemaField("close", "float", mode="REQUIRED"),
    bigquery.SchemaField("volume", "integer", mode="REQUIRED"),
    bigquery.SchemaField("day", "integer", mode="REQUIRED"),
    bigquery.SchemaField("month", "integer", mode="REQUIRED"),
    bigquery.SchemaField("year", "integer", mode="REQUIRED"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, exists_ok=True)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)
